https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query=%EC%9E%90%EC%86%8C%EC%84%9C&oquery=&tqi=hXADkdp0JXVssKHZiFNssssssd8-338598

https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%9E%90%EC%86%8C%EC%84%9C&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=16&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=11

In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
from regex import F
import requests
import pandas as pd
import re

In [2]:
def day(week):
    now = datetime.now()
    e_date = now - timedelta(weeks=week)
    #s_date = s_date.strftime("%Y.%m.%d")
    e_date = e_date.strftime("%Y.%m.%d")

    return e_date

In [3]:
maxpage = '300'
query = '제이오텍'
sort = '0'
s_date = day(0) # 0주 오늘
e_date = day(52) # 52주
osc = ''
nos = ''

In [4]:
def NaverNews(maxpage, query, sort, s_date, e_date, osc, nos):
    title_text=[]
    link_text=[]
    source_text=[]
    date_text=[]
    contents_text=[]
    result={}

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    def date_cleansing(test):
        try:
            #지난 뉴스
            #머니투데이  10면1단  2018.11.05.  네이버뉴스   보내기
            pattern = '\d+.(\d+).(\d+).'  #정규표현식
        
            r = re.compile(pattern)
            match = r.search(test).group(0)  # 2018.11.05.
            date_text.append(match)
            #return match
            
        except AttributeError:
            #최근 뉴스
            #이데일리  1시간 전  네이버뉴스   보내기  
            pattern = '\w* (\d\w*)'     #정규표현식 
            
            r = re.compile(pattern)
            match = r.search(test).group(1)
            date_text.append(match)
            #return match

    def contents_cleansing(contents):
        first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                        str(contents)).strip()  #앞에 필요없는 부분 제거
        second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                        first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
        third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
        contents_text.append(third_cleansing_contents)
        #return contents_text

    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query +"&sm=tab_opt&sort="+ sort + "&photo=0&field=0&pd=3" +"&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=0&office_type=" + '&office_section_code='+ osc + '&news_office_checked='+ nos + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0" + "%2Ca%3A&start=" + str(page)
        # office_section_code=11&news_office_checked=1143
        
        response = requests.get(url)
        html = response.text

        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')

        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)     #제목
            link_text.append(atag['href'])   #링크주소
            
        #신문사 추출
        source_lists = soup.select('.info_group > .press')
        for source_list in source_lists:
            source_text.append(source_list.text)    #신문사
        
        #날짜 추출 
        date_lists = soup.select('.info_group > span.info')
        for date_list in date_lists:
            # 1면 3단 같은 위치 제거
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)
        
        #본문요약본
        contents_lists = soup.select('.news_dsc')
        for contents_list in contents_lists:
            contents_cleansing(contents_list) #본문요약 정제화
        

        #모든 리스트 딕셔너리형태로 저장
        result= {"date" : date_text , "title":title_text , "source" : source_text , "contents": contents_text ,"link":link_text }
        #print(page)
        
        df = pd.DataFrame(result)  #df로 변환
        page += 10

    return df

In [5]:
keyword = NaverNews(maxpage, query, sort, s_date, e_date, osc, nos)

In [6]:
keyword = keyword.drop_duplicates(['title'], keep=False, ignore_index=True)

In [7]:
keyword

,date,title,source,contents,link
0,5일 전,"바이러스 실험, 난류 없이 안전하게",산업일보,제이오텍 남형욱 PM은 장비에 관해 “작업장 공기와 기기 내부 공기를 분리해 사용자...,http://www.kidd.co.kr/news/228710
1,14시간 전,대전 연구산업협의회 발족…300여개 기업·기관 뭉쳤다,대전일보,"이 외에도 MKE, 알씨엔, 비엠테크, 한국과학기술지주, 비즈니스전략연구소, 동광,...",http://www.daejonilbo.com/news/articleView.htm...
2,2022.03.18.,"제이오텍, InterBattery 2022서 자동 소화장치 구비한 배터리 시험 챔버...",에이빙뉴스,InterBattery 2022에서 제이오텍이 선보인 배터리 시험 챔버 │촬영-에이...,http://kr.aving.net/news/view.php?articleId=16...
3,2022.03.02.,"원텍·제이오텍·제노포커스·켐옵틱스, 지역혁신 기업 선정",헬로디디,정부가 지역경제 혁신성장을 이끌 지역혁신 선도기업 54개를 선정한 가운데 대전은 원...,http://www.hellodd.com/news/articleView.html?i...
4,2021.11.01.,"제이오텍, KOREA LIFE SCIENCE WEEK 2021 참가... 바이러스 ...",에이빙뉴스,사진 제공-제이오텍 ㈜제이오텍은 오는 11월 3일(수)부터 4일(목)까지 양일간 서...,http://kr.aving.net/news/view.php?articleId=16...
5,2022.02.18.,"원텍·제노포커스·제이오텍·켐옵틱스, 대전시 혁신 선도기업 선정",벤처스퀘어,대전시는 지역산업 성장과 경제 활성화를 이끌 ‘지역혁신 선도기업’으로 ▲원텍㈜ ▲㈜...,https://www.venturesquare.net/848982
6,2021.09.28.,"제이오텍, 제조부터 출하까지 품질관리…'2~8도 유지' 백신냉장고 출시",한국경제,‘연구 실험인의 동반자’를 표방하는 제이오텍은 최근 코로나19 백신 보관을 위해 영...,https://www.hankyung.com/economy/article/20210...
7,2021.11.05.,"국산 실험 장비, 더 스마트해진다…작업자 보호·IoT 기능 탑재까지",산업일보,"참가한 ㈜제이오텍(JEIO TECH)은 연구용 실험기기 및 작업대, 산업용 오븐 및...",http://www.kidd.co.kr/news/224758
8,2021.11.07.,[2021 대전시 유망중소기업] 새로운 세상으로 발돋움하는 중소기업의 도전,금강일보,■㈜제이오텍 www.jeiotech.com ㈜제이오텍(대표 신현주·김기성)은 지난 ...,http://www.ggilbo.com/news/articleView.html?id...
9,2021.09.29.,"[조간브리핑]초소형 위성부터 진공단열재까지, 미래 제조업 '유니콘' 여기에!",헬로디디,"제이오텍, 제조부터 출하까지 품질관리…'2~8도 유지' 백신냉장고 출시 '연구 실험...",https://www.hellodd.com/news/articleView.html?...


In [8]:
now = datetime.now()

outputFileName = 'News_%s.csv' % (query)
RESULT_PATH = 'C:/Django/Data_Processing/Data/'

keyword.to_csv(RESULT_PATH + outputFileName, encoding='utf-8-sig', index=False)

In [10]:
import sqlite3

In [11]:
con = sqlite3.connect("C:/Django/test.db")

In [15]:
keyword.to_sql('news_제이오텍',con)

21

In [16]:
con.commit()